In [88]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np
from src import parse
from collections import Counter

import logging
import sys


log = logging.getLogger()
log.setLevel(logging.DEBUG)

if not log.hasHandlers():
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    log.addHandler(handler)

/Users/andrew/src/otp-scheduler


In [89]:
log.info('hi')

INFO - hi


In [90]:
# ASSUMPTIONS
MEAL_CAPACITY = 27500
MAX_DROPOFFS_PER_RUN = 4
MIN_MEALS_PER_RUN = 50

# This can be set a value from 0 (no effect) to the total count of restaurants (large effect)
RESTAURANT_FOUNDER_KNOB = 0

curr_capacity = MEAL_CAPACITY

def sort_schedule(df):
    return df.sort_values(by=['zone', 'priority_rank', 'code']).reset_index(drop=True)

restaurants, num_zones = parse.parse_from_restaurants_csv('../../examples/boston/20200427/restaurants.csv')
restaurants.set_index('code', inplace=True)
hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200427/hospitals.csv')).set_index('name')
vip_hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200427/vips.csv')).set_index('name')
vip_schedule = vip_hospitals.iloc[:, 4:]

assert hospitals['zone'].max() <= num_zones, 'A hospital is in a zone that we didn\'t expect from the restaurant sheet!'

rest_usage = restaurants.copy()
rest_usage.iloc[:, 3:(rest_usage.shape[1] - num_zones)] = 0

founding_restaurants = restaurants[restaurants['founder'] == 1].index.values

rest_cap = restaurants.copy()

total_hospitals = hospitals.shape[0]

print('Number of Restaurants:', restaurants.shape[0])
print('Founding Restaurants:', founding_restaurants.tolist())
print('Number of Hospitals:', total_hospitals)
print('Number of Zones:', num_zones)

Number of Restaurants: 18
Founding Restaurants: ['PG']
Number of Hospitals: 45
Number of Zones: 5


In [91]:
def day_from_meal(meal):
    return meal.replace('lunch', '').replace('breakfast', '').replace('dinner', '').replace('_', '')

meal_log = None
meal_log_key_kludge = set()

def get_meal_log(schedule, is_vip):
    log = []
    for name, row in schedule.iterrows():
        for j, val in enumerate(row[4:]):
            # Add only if this cell is not blank, AND we haven't already added this hospital/meal.
            # This is possible if this order exists in both the regular and VIP meal schedule.
            # Since we add the VIP meals first, then we're guaranteed for it to be counted as a VIP meal if it's in both places.
            if not pd.isnull(val) and (name, schedule.columns[j + 4]) not in meal_log_key_kludge:
                log.append([
                    name,
                    row['zone'],
                    row['priority_rank'],
                    is_vip,
                    schedule.columns[j + 4],
                    val, 
                    np.nan,
                    day_from_meal(schedule.columns[j + 4]),
                    False
                ])
                meal_log_key_kludge.add((name, schedule.columns[j + 4]))
    return pd.DataFrame(log, columns=['hospital', 'zone', 'priority_rank', 'vip', 'meal', 'quantity', 'restaurant', 'day', 'failed'])

vip_log = get_meal_log(vip_hospitals, True)
meal_log = get_meal_log(hospitals, False).append(vip_log)

In [92]:
log.info(meal_log.shape)

INFO - (362, 9)


In [93]:
def get_rest_cap_cluster_for_meal(meal, zone):
    z = 'zone_' + str(zone)
    r = rest_cap[rest_cap[z] > 0][[meal, z]]
    return r

In [94]:
def get_oustanding_vip_meals():
    return meal_log[(meal_log['vip'] == True) & meal_log['restaurant'].isnull()]

def get_outstanding_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone) & meal_log['restaurant'].isnull()]

def get_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone)]

def is_restaurant_serving_same_day_other_meal(rest_code, meal):
    day = day_from_meal(meal)
    rest_serving = meal_log[(meal_log['restaurant'] == rest_code) & (meal_log['day'] == day) & meal_log['meal'] != meal]
    return not rest_serving.empty

def assign_hospital_order(row):
    global curr_capacity

    meal = row['meal']
    hospital = row['hospital']
    zone = row['zone']
    
    print('Assigning', hospital, meal)
    
    # Check to see if we've already done this (sanity)
    if not pd.isnull(row['restaurant']):
        print('We have already assigned this one!')
        return False
        
    hospitals_in_cluster = get_hospitals_in_cluster_for_meal(meal, zone)

    # Restaurant capacity
    r_cap = get_rest_cap_cluster_for_meal(meal, zone)
        
    # The amount have to serve
    target_quantity = row['quantity']
    
    if curr_capacity - target_quantity < 0:
        print('We cannot to afford this meal because of our global limit!', curr_capacity, target_quantity)
        return False
    
    print(hospital, meal, zone, target_quantity)
    
    def find_restaurant():
        # Check if a restaurant is already serving the cluster
        restaurants_serving_cluster = hospitals_in_cluster[hospitals_in_cluster['restaurant'].notnull()]['restaurant']
        
        if not restaurants_serving_cluster.empty:
            # Filter out any that are already at the dropout limit
            rest_counter = Counter(restaurants_serving_cluster)
            rest_cluster_keys = rest_counter.keys()
            print('Checking for restaurants already in cluster', rest_counter)
            rest_with_room = [c[0] for c in rest_counter.most_common() if c[1] < MAX_DROPOFFS_PER_RUN]
            
            # Find restaurants with capacity
            candidates_with_quantity = []
            for r in rest_with_room:
                q = rest_cap.loc[r, meal]
                if q >= target_quantity:
                    print('Found an existing restaurant with capacity', r, q)
                    candidates_with_quantity.append(r)
                    if is_restaurant_serving_same_day_other_meal(r, meal):
                        print('Returning a restaurant with capacity, working on the same day', r, q)
                        return r
            if len(candidates_with_quantity):
                print('Returning a restaurant with capacity', r, q)
                return candidates_with_quantity[0]
        
        # Couldn't find an appropriate restaurant already serving cluster
        print('No suitable restaurants already working this cluster')
        
        # Find restaurants that are already serving the same meal time but in a different zone
        restaurants_in_other_zones = meal_log[
            (meal_log['meal'] == meal) & (
                meal_log['zone'] != zone) & (
                meal_log['restaurant'].notnull())]['restaurant'].unique()
                
        print('filtering down candidates with three lists', restaurants_serving_cluster, restaurants_in_other_zones)
        r_candidates = r_cap[
            (~r_cap.index.isin(restaurants_serving_cluster)) & (
                ~r_cap.index.isin(restaurants_in_other_zones)) & (
                r_cap[meal] >= target_quantity)
        ]
        
        print('Number of candidates with capacity', r_candidates.shape[0])
        
        if r_candidates.empty:
            return None
        
        # TODO add is_restaurant_serving_same_day_other_meal logic down here too!
        
        zone_col = 'zone_' + str(zone)

        # Sort in zone preference + quantity order (TODO KNOB: THIS USED TO BE ascending=False)
        r_candidates = r_candidates.sort_values(by=[zone_col, meal], ascending=[False, True])
        
        day = day_from_meal(meal)
        
        # TODO Make sure restaurant isn't overbooked on their max day count
        final_valid_list = []
        for i, r in r_candidates.iterrows():
            print(i)
            days_worked = set(meal_log[meal_log['restaurant'] == i]['day'])
            # Check if we're already working this day, or the new day is still under cap
            if day in days_worked or len(days_worked) < rest_cap.loc[i, 'max_days_per_week']:
                # return i
                final_valid_list.append(i)
        
        # MEAL STEAL -- We need to prioritize founding Restaurants (like "PG, LD" in the Boston case)
        # TODO -- parameterize this, both the restaurants we want to do this for, and also how much we crank this knob
        # Comment out this section to ignore
        if len(final_valid_list) == 0:
            return None
        else:
            if len(final_valid_list) == 1:
                return final_valid_list[0]
            else:
                for r in final_valid_list[0:RESTAURANT_FOUNDER_KNOB]:
                    if r in founding_restaurants:
                        return r
                return final_valid_list[0]
        
        return None
          
    r_code = find_restaurant()
    
    if r_code is None:
        print('CANNOT SERVE HOSPITAL FOR THEIR REQUEST', hospital, meal, r_cap)
        return False
                
    # Allocate the restaurant to the hospital(s)
    print('debiting r capacity: ', r_code, rest_cap.loc[r_code, meal])
    rest_cap.loc[r_code, meal] -= target_quantity
            
    print('increasing r usage', r_code, target_quantity)
    rest_usage.loc[r_code, meal] += target_quantity
        
    curr_capacity -= target_quantity
                
    print('updating log', hospital, meal, r_code)
    meal_log.loc[(meal_log['hospital'] == hospital) & (meal_log['meal'] == meal), 'restaurant'] = r_code
    
    return True

# HANDLE VIPS
for i, row in get_oustanding_vip_meals().iterrows():
    assigned = assign_hospital_order(row)
    if not assigned:
        meal_log.loc[(meal_log['hospital'] == row['hospital']) & (meal_log['meal'] == row['meal']), 'failed'] = True
    print()
    print() 

Assigning Brigham and Women's Hospital - ED lunch_wed
Brigham and Women's Hospital - ED lunch_wed 1 100.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: float64) []
Number of candidates with capacity 15
ME
RT
CE
FV
SH
BB
MY
PO
BC
RC
CL
MD
LD
SU
PG
debiting r capacity:  ME 100.0
increasing r usage ME 100.0
updating log Brigham and Women's Hospital - ED lunch_wed ME


Assigning Brigham and Women's Hospital - ED lunch_sun
Brigham and Women's Hospital - ED lunch_sun 1 100.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 8
SH
BB
MY
CL
MD
LD
SU
PG
debiting r capacity:  SH 200.0
increasing r usage SH 100.0
updating log Brigham and Women's Hospital - ED lunch_sun SH


Assigning Brigham and Women's Hospital - ED dinner_wed
Brigham and Women's Hospital - ED dinner_wed 1 100.0
No suit

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)



MY
CL
MD
LD
SU
PG
debiting r capacity:  BB 200.0
increasing r usage BB 200.0
updating log Brigham & Women's Hospital (COVID ICUs) lunch_sun BB


Assigning Brigham & Women's Hospital (COVID ICUs) dinner_mon
Brigham & Women's Hospital (COVID ICUs) dinner_mon 1 210.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 6
BC
RC
MD
LD
SU
PG
debiting r capacity:  BC 300.0
increasing r usage BC 210.0
updating log Brigham & Women's Hospital (COVID ICUs) dinner_mon BC


Assigning Brigham & Women's Hospital (COVID ICUs) dinner_tues
Brigham & Women's Hospital (COVID ICUs) dinner_tues 1 210.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 6
BC
RC
MD
LD
SU
PG
debiting r capacity:  BC 300.0
increasing r usage BC 210.0
updating log Brigham & Women'

filtering down candidates with three lists Series([], Name: restaurant, dtype: object) ['PO' 'SH']
Number of candidates with capacity 11
MY
RC
SU
RT
CE
BB
BC
CL
MD
LD
PG
debiting r capacity:  MY 200.0
increasing r usage MY 12.0
updating log Boston Healthcare for the Homeless - Tents lunch_mon MY


Assigning Boston Healthcare for the Homeless - Tents lunch_tues
Boston Healthcare for the Homeless - Tents lunch_tues 2 12.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) ['PO']
Number of candidates with capacity 11
MY
SU
ME
CE
SH
BB
BC
CL
MD
LD
PG
debiting r capacity:  MY 200.0
increasing r usage MY 12.0
updating log Boston Healthcare for the Homeless - Tents lunch_tues MY


Assigning Boston Healthcare for the Homeless - Tents lunch_wed
Boston Healthcare for the Homeless - Tents lunch_wed 2 12.0
Checking for restaurants already in cluster Counter({'MY': 1})
Found an existing restaurant with capacity


Assigning BWH - Faulkner 7N Covid Unit dinner_mon
BWH - Faulkner 7N Covid Unit dinner_mon 2 34.0
Checking for restaurants already in cluster Counter({'RC': 1})
Found an existing restaurant with capacity RC 288.0
Returning a restaurant with capacity, working on the same day RC 288.0
debiting r capacity:  RC 288.0
increasing r usage RC 34.0
updating log BWH - Faulkner 7N Covid Unit dinner_mon RC


Assigning BWH - Faulkner 7N Covid Unit dinner_tues
BWH - Faulkner 7N Covid Unit dinner_tues 2 34.0
Checking for restaurants already in cluster Counter({'RC': 1})
Found an existing restaurant with capacity RC 288.0
Returning a restaurant with capacity, working on the same day RC 288.0
debiting r capacity:  RC 288.0
increasing r usage RC 34.0
updating log BWH - Faulkner 7N Covid Unit dinner_tues RC


Assigning BWH - Faulkner 7N Covid Unit dinner_wed
BWH - Faulkner 7N Covid Unit dinner_wed 2 34.0
Checking for restaurants already in cluster Counter({'RC': 2})
Found an existing restaurant with capa

Returning a restaurant with capacity, working on the same day SU 439.0
debiting r capacity:  SU 439.0
increasing r usage SU 20.0
updating log Boston Healthcare for the Homeless - Albany Street dinner_fri SU


Assigning Boston Healthcare for the Homeless - Albany Street dinner_sat
Boston Healthcare for the Homeless - Albany Street dinner_sat 2 20.0
Checking for restaurants already in cluster Counter({'SU': 4})
No suitable restaurants already working this cluster
filtering down candidates with three lists 43    SU
50    SU
64    SU
78    SU
Name: restaurant, dtype: object ['SH' 'RC']
Number of candidates with capacity 5
MM
BB
MD
LD
PG
debiting r capacity:  MM 120.0
increasing r usage MM 20.0
updating log Boston Healthcare for the Homeless - Albany Street dinner_sat MM


Assigning Boston Healthcare for the Homeless - Albany Street dinner_sun
Boston Healthcare for the Homeless - Albany Street dinner_sun 2 20.0
Checking for restaurants already in cluster Counter({'SU': 4, 'BB': 1})
Found an

Boston EMS - South End breakfast_tues 3 7.0
Checking for restaurants already in cluster Counter({'BB': 1})
Found an existing restaurant with capacity BB 370.0
Returning a restaurant with capacity, working on the same day BB 370.0
debiting r capacity:  BB 370.0
increasing r usage BB 7.0
updating log Boston EMS - South End breakfast_tues BB


Assigning Boston EMS - South End breakfast_wed
Boston EMS - South End breakfast_wed 3 25.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 2
CL
BB
debiting r capacity:  BB 400.0
increasing r usage BB 25.0
updating log Boston EMS - South End breakfast_wed BB


Assigning Boston EMS - South End lunch_mon
Boston EMS - South End lunch_mon 3 25.0
Checking for restaurants already in cluster Counter({'RT': 1})
Found an existing restaurant with capacity RT 70.0
Returning a restaurant with capacity, working on the same day RT 70.0


In [95]:
meal_log[(meal_log.vip == True) & (meal_log.failed == True)]

,hospital,zone,priority_rank,vip,meal,quantity,restaurant,day,failed


In [96]:
hospitals_that_failed_placement = set()

def select_and_assign_hospital():
    next_hospital = meal_log[~meal_log.hospital.isin(list(hospitals_that_failed_placement))].groupby('hospital')[['priority_rank', 'restaurant']].agg({
            'restaurant': 'count',
            'priority_rank': 'min'
        }).sort_values(by=['restaurant', 'priority_rank']).reset_index()
    
    if next_hospital.shape[0] > 0:
        next_hospital = next_hospital.iloc[0]
    else:
        return False
    
    print('Next hospital:', next_hospital)

    # Find potential orders to serve, ordered by quantity
    possible_orders = meal_log[(meal_log['hospital'] == next_hospital['hospital']) & (meal_log['restaurant'].isnull()) & (meal_log['failed'] == False)].sort_values(by='quantity', ascending=False)
    
    print('Possible orders', possible_orders)
    for i, row in possible_orders.iterrows():
        if assign_hospital_order(row):
            return True
        else:
            meal_log.loc[
                (meal_log['hospital'] == row['hospital']) & (meal_log['meal'] == row['meal']), 
                'failed'
            ] = True
            
    hospitals_that_failed_placement.add(next_hospital['hospital'])
    return True
    

while curr_capacity > 0 and len(hospitals_that_failed_placement) < total_hospitals:
    retval = select_and_assign_hospital()
    if retval == False:
        break
    print()
    print()

Next hospital: hospital         Boston Children's Hospital - ED
restaurant                                     0
priority_rank                                  1
Name: 0, dtype: object
Possible orders                           hospital  zone  priority_rank    vip          meal  \
0  Boston Children's Hospital - ED     1              1  False    dinner_mon   
1  Boston Children's Hospital - ED     1              1  False   dinner_tues   
2  Boston Children's Hospital - ED     1              1  False    dinner_wed   
3  Boston Children's Hospital - ED     1              1  False  dinner_thurs   
4  Boston Children's Hospital - ED     1              1  False    dinner_fri   

   quantity restaurant    day  failed  
0      60.0        NaN    mon   False  
1      60.0        NaN   tues   False  
2      60.0        NaN    wed   False  
3      60.0        NaN  thurs   False  
4      60.0        NaN    fri   False  
Assigning Boston Children's Hospital - ED dinner_mon
Boston Children's Hospita

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


 Steward St. Elizabeth's Medical Center lunch_mon CE


Next hospital: hospital         Tufts Medical Center
restaurant                          0
priority_rank                       1
Name: 0, dtype: object
Possible orders                  hospital  zone  priority_rank    vip          meal  quantity  \
201  Tufts Medical Center     3              1  False    dinner_mon      54.0   
202  Tufts Medical Center     3              1  False   dinner_tues      54.0   
203  Tufts Medical Center     3              1  False    dinner_wed      54.0   
204  Tufts Medical Center     3              1  False  dinner_thurs      54.0   
205  Tufts Medical Center     3              1  False    dinner_fri      54.0   
196  Tufts Medical Center     3              1  False     lunch_mon      50.0   
197  Tufts Medical Center     3              1  False    lunch_tues      50.0   
198  Tufts Medical Center     3              1  False     lunch_wed      50.0   
199  Tufts Medical Center     3              1  

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_value


Assigning Dimock Center lunch_tues
Dimock Center lunch_tues 2 20.0
Checking for restaurants already in cluster Counter({'MY': 4, 'SU': 1})
Found an existing restaurant with capacity SU 100.0
Returning a restaurant with capacity, working on the same day SU 100.0
debiting r capacity:  SU 100.0
increasing r usage SU 20.0
updating log Dimock Center lunch_tues SU


Next hospital: hospital         Dot House Health Dorchester
restaurant                                 0
priority_rank                              2
Name: 0, dtype: object
Possible orders                        hospital  zone  priority_rank    vip         meal  \
61  Dot House Health Dorchester     2              2  False    lunch_mon   
62  Dot House Health Dorchester     2              2  False   lunch_tues   
63  Dot House Health Dorchester     2              2  False    lunch_wed   
64  Dot House Health Dorchester     2              2  False  lunch_thurs   
65  Dot House Health Dorchester     2              2  False    lunc

...

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_value

Next hospital: hospital         Brigham & Women's Hospital (COVID ICUs)
restaurant                                            15
priority_rank                                          1
Name: 0, dtype: object
Possible orders                                    hospital  zone  priority_rank    vip  \
18  Brigham & Women's Hospital (COVID ICUs)     1              1  False   

             meal  quantity restaurant  day  failed  
18  breakfast_sun     200.0        NaN  sun   False  
Assigning Brigham & Women's Hospital (COVID ICUs) breakfast_sun
Brigham & Women's Hospital (COVID ICUs) breakfast_sun 1 200.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) ['BB' 'CL']
Number of candidates with capacity 0
CANNOT SERVE HOSPITAL FOR THEIR REQUEST Brigham & Women's Hospital (COVID ICUs) breakfast_sun       breakfast_sun  zone_1
code                       
PG              NaN     0.5
LD              NaN    

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_value

 breakfast_sun 2 12.0
Checking for restaurants already in cluster Counter({'BB': 2})
Found an existing restaurant with capacity BB 377.0
Returning a restaurant with capacity, working on the same day BB 377.0
debiting r capacity:  BB 377.0
increasing r usage BB 12.0
updating log Boston Healthcare for the Homeless - Tents breakfast_sun BB


Next hospital: hospital         Pine Street Inn - Site 1
restaurant                             16
priority_rank                           2
Name: 0, dtype: object
Possible orders                      hospital  zone  priority_rank    vip         meal  \
120  Pine Street Inn - Site 1     2              2  False  lunch_thurs   
121  Pine Street Inn - Site 1     2              2  False    lunch_fri   

     quantity restaurant    day  failed  
120      15.0        NaN  thurs   False  
121      15.0        NaN    fri   False  
Assigning Pine Street Inn - Site 1 lunch_thurs
Pine Street Inn - Site 1 lunch_thurs 2 15.0
Checking for restaurants already in clu

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [97]:
curr_capacity

329.0

In [98]:
meal_log[meal_log['restaurant'].notnull()]['quantity'].sum()

27171.0

In [99]:
# Create empty results frame
output = hospitals.copy()
output.iloc[:, 4:] = np.nan


# construct the double-wide output dataframe
for i, c in enumerate(hospitals.columns[4:]):
    output.insert((i*2+1)+4, c + '_rest', np.nan)
        
for i, row in meal_log.iterrows():
    output.loc[row['hospital'], row['meal']] = row['quantity']
    if not pd.isnull(row['restaurant']):
        output.loc[row['hospital'], row['meal'] + '_rest'] = row['restaurant']
    else:
        output.loc[row['hospital'], row['meal'] + '_rest'] = 'XXX'


# for i, row in meal_log.iterrows():
#     if not pd.isnull(row['restaurant']):
#         val = str(row['restaurant'] + ' - ' + str(row['quantity']))
#     else:
#         val = str('XXX - ' + str(row['quantity']))
#     output.loc[row['hospital'], row['meal']] = val
    
# for i, row in meal_log.iterrows():
#     if not pd.isnull(row['restaurant']):
#         output.loc[row['hospital'], row['meal']] = row['restaurant']
        
output.to_csv('../../output.csv')


In [100]:
output

,code,priority_rank,region,zone,breakfast_mon,breakfast_mon_rest,breakfast_tues,breakfast_tues_rest,breakfast_wed,breakfast_wed_rest,...,dinner_wed,dinner_wed_rest,dinner_thurs,dinner_thurs_rest,dinner_fri,dinner_fri_rest,dinner_sat,dinner_sat_rest,dinner_sun,dinner_sun_rest
name,,,,,,,,,,,,,,,,,,,,,
Brigham and Women's Hospital - ED,4,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,CS,NaN,NaN,NaN,NaN,NaN,NaN,100.0,MM
Boston Children's Hospital - ED,18,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,60.0,BC,60.0,BC,60.0,BC,NaN,NaN,NaN,NaN
Dana Farber Cancer Institute - Inpatient PAs,23,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brigham & Women's Hospital (COVID ICUs),36,1,Longwood / Back Bay,1,210.0,XXX,210.0,XXX,210.0,XXX,...,210.0,BC,210.0,BC,210.0,BC,200.0,SH,200.0,SH
Brigham and Women's Hospital - Central,52,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,300.0,RC,300.0,MD
Beth Israel Deaconess Medical Center - East Campus,53,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Beth Israel Deaconess Medical Center - West Campus,54,1,Longwood / Back Bay,1,NaN,NaN,150.0,CL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Whittier Street Health Center,15,2,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BWH - Faulkner ED,5,1,Roxbury / Jamaica Plain,2,NaN,NaN,NaN,NaN,NaN,NaN,...,45.0,RC,NaN,NaN,NaN,NaN,NaN,NaN,45.0,SU


In [111]:
rest_usage.iloc[:, 3:(rest_usage.shape[1] - num_zones)].sum(axis=1).sort_values(ascending=False).sum()

27171.0

In [112]:
hospitals.iloc[:, 4:].sum().sum()

28811.0

In [103]:
equity = meal_log.groupby(['priority_rank', 'hospital'])[['meal', 'vip', 'restaurant']].agg({
    'restaurant': [('orders_fulfilled', 'count')],
    'meal': [('orders_requested', 'count')],
    'vip': [('vip_orders', 'sum')],
})
equity

restaurant  \
                                                                 orders_fulfilled   
priority_rank hospital                                                              
1             BWH - Faulkner 7N Covid Unit                                     14   
              BWH - Faulkner ED                                                 4   
              BWH - Faulkner Hospital Central                                   2   
              BWH - Faulkner ICU                                               14   
              Beth Israel Deaconess Medical Center - East Campus                2   
              Beth Israel Deaconess Medical Center - West Campus                3   
              Boston Children's Hospital - ED                                   5   
              Boston EMS - South End                                            8   
              Boston Healthcare for the Homeless - Albany Street               14   
              Boston Healthcare for the Homeless - Tents                       17   
              Boston Medical Center                                            14   
              Brigham & Women's Hospital (COVID ICUs)                          15   
              Brigham and Women's Hospital - Central                            4   
              Brigham and Women's Hospital - ED                                 4   
              Cambridge Health Alliance - Cambridge Hospital                    7   
              Cambridge Health Alliance - Everett Hospital                      7   
              Carney Hospital - ED                                             14   
              Dana Farber Cancer Institute - Inpatient PAs                      7   
              Massachusetts General Hospital - Central                          7   
              Massachusetts General Hospital - PICU                            10   
              Mt. Auburn Hospital                                               1   
              Newton Wellesley Hospital                                         1   
              Steward St. Elizabeth's Medical Center                           10   
              Tufts Medical Center                                             10   
              VA Medical Center - West Roxbury                                  6   
2             BIDMC - Chelsea                                                   5   
              Bowdoin Street Health Center                                      3   
              Charles River Community Health                                    5   
              Codman Square Health Center - Site 1                              5   
              Codman Square Health Center - Site 2                              5   
              Dimock Center                                                     4   
              Dot House Health Dorchester                                       5   
              East Boston Neighborhood Health Center                            6   
              Harbor Health Services Inc.                                       2   
              Harvard Street Neighborhood Health Center                         5   
              Mattapan Community Health Center                                  3   
              Pine Street Inn - Site 1                                         18   
              Pine Street Inn - Site 2                                         17   
              Pine Street Inn Holy Family Shelter                              12   
              Pine Street Inn Shattuck Shelter                                 16   
              South Boston Community Health Center                              5   
              South End Community Health Center                                 5   
              Southern Jamaica Plain Health Center (part of B...                5   
              Upham's Corner Health Center                                      5   
              Whittier Street Health Center                                 

In [104]:
equity.to_csv('../../equity.csv')

In [105]:
meal_log[meal_log['hospital'] == 'Boston Medical Center']

,hospital,zone,priority_rank,vip,meal,quantity,restaurant,day,failed
51,Boston Medical Center,2,1,False,lunch_sat,150.0,SU,sat,False
52,Boston Medical Center,2,1,False,lunch_sun,150.0,LD,sun,False
53,Boston Medical Center,2,1,False,dinner_mon,400.0,SU,mon,False
54,Boston Medical Center,2,1,False,dinner_tues,400.0,SU,tues,False
55,Boston Medical Center,2,1,False,dinner_wed,400.0,PG,wed,False
56,Boston Medical Center,2,1,False,dinner_thurs,400.0,MD,thurs,False
57,Boston Medical Center,2,1,False,dinner_fri,400.0,MD,fri,False
45,Boston Medical Center,2,1,True,lunch_mon,400.0,SU,mon,False
46,Boston Medical Center,2,1,True,lunch_tues,400.0,SU,tues,False
47,Boston Medical Center,2,1,True,lunch_wed,400.0,SU,wed,False


In [106]:
meal_log[meal_log['hospital'] == 'Massachusetts General Hospital - Central']

,hospital,zone,priority_rank,vip,meal,quantity,restaurant,day,failed
106,Massachusetts General Hospital - Central,3,1,True,lunch_tues,500.0,LD,tues,False
107,Massachusetts General Hospital - Central,3,1,True,lunch_thurs,500.0,LD,thurs,False
108,Massachusetts General Hospital - Central,3,1,True,lunch_sat,300.0,CL,sat,False
109,Massachusetts General Hospital - Central,3,1,True,dinner_mon,500.0,LD,mon,False
110,Massachusetts General Hospital - Central,3,1,True,dinner_wed,500.0,LD,wed,False
111,Massachusetts General Hospital - Central,3,1,True,dinner_fri,500.0,LD,fri,False
112,Massachusetts General Hospital - Central,3,1,True,dinner_sun,300.0,LD,sun,False


In [107]:
rest_usage.iloc[:, 3:-num_zones].sum(axis=1).sort_values(ascending=False)

code
SU    4751.0
LD    3545.0
BC    2737.0
MD    2560.0
PG    2288.0
BB    2157.0
SH    1975.0
CL    1746.0
RC    1677.0
PO    1200.0
MY     929.0
ME     460.0
CE     368.0
RT     221.0
MM     199.0
FV     184.0
CS     174.0
FC       0.0
dtype: float64

In [108]:
rest_stats = meal_log[['restaurant', 'quantity']].groupby('restaurant').agg({
    'quantity': ['sum', 'count', 'mean', 'min', 'max']
})
rest_stats.columns = ['sum_total', 'count', 'mean', 'min', 'max']
rest_stats = rest_stats.sort_values(by='sum_total', ascending=False)
rest_stats

,sum_total,count,mean,min,max
restaurant,,,,,
SU,4751.0,56,84.839286,4.0,400.0
LD,3545.0,23,154.130435,7.0,500.0
BC,2737.0,21,130.333333,12.0,225.0
MD,2560.0,23,111.304348,7.0,400.0
PG,2288.0,14,163.428571,4.0,550.0
BB,2157.0,45,47.933333,3.0,200.0
SH,1975.0,17,116.176471,7.0,200.0
CL,1746.0,23,75.913043,3.0,300.0
RC,1677.0,33,50.818182,10.0,300.0


In [109]:
rest_stats.to_csv('../../rest_stats.csv')